In [2]:
# !pip install pyDecision # Uncomment this line for installation
import numpy as np
from pyDecision.algorithm import electre_iii

In [3]:
# =========================
# 1. Matriz de decisión
# =========================
matrix = np.array([
    [80, 50, 120],   # A
    [75, 40, 150],   # B
    [85, 55, 110]    # C
])

# =========================
# 2. Pesos (deben sumar 1)
# =========================
weights = np.array([0.4, 0.35, 0.25])

# =========================
# 3. Tipos de criterio
# g1 = Beneficio (max)
# g2 = Costo (min)
# g3 = Latencia (min)
types = ['max', 'min', 'min']
# =========================
#types = np.array([1, -1, -1])

# =========================
# 4. Umbrales de ELECTRE III
# q = indiferencia
# p = preferencia
# v = veto
# =========================
q = np.array([2, 3, 5])   # Indiferencia
p = np.array([6, 8, 15])     # Preferencia
v = np.array([15, 20, 40])     # Veto

# =========================
# 5. Ejecutar ELECTRE III
# =========================
# Orden correcto:
result = electre_iii(matrix, p, q, v, weights, graph=False)

# =========================
# 6. Mostrar resultado
# =========================
print("Resultado:")
print(result)

Resultado:
(array([[1.   , 0.75 , 0.56 ],
       [0.35 , 1.   , 0.25 ],
       [0.875, 0.75 , 1.   ]]), array([[0.        , 0.75      , 0.56      ],
       [0.35      , 0.        , 0.10288066],
       [0.875     , 0.        , 0.        ]]), ['a3', 'a1', 'a2'], ['a3', 'a1', 'a2'], ['a1', 'a2', 'a3'], array([['-', 'P+', 'P-'],
       ['P-', '-', 'P-'],
       ['P+', 'P+', '-']], dtype='<U25'))


In [4]:
global_conc, credibility, rank_D, rank_A, rank_M, rank_P = result

print("Ranking descendente (D):", rank_D)
print("Ranking ascendente (A):", rank_A)
print("Ranking final recomendado:", rank_D)  # si D y A coinciden


Ranking descendente (D): ['a3', 'a1', 'a2']
Ranking ascendente (A): ['a3', 'a1', 'a2']
Ranking final recomendado: ['a3', 'a1', 'a2']


In [5]:
ranking_final = rank_D if rank_D == rank_A else rank_M


In [6]:
ranking_final

['a3', 'a1', 'a2']

In [7]:
print("Matriz de credibilidad:")
print(credibility)

Matriz de credibilidad:
[[0.         0.75       0.56      ]
 [0.35       0.         0.10288066]
 [0.875      0.         0.        ]]
